In [14]:
import json

import numpy as np
import torch
from torch import nn, optim

from collections import defaultdict

from torch.utils import data
from trajdata import AgentBatch, AgentType, UnifiedDataset
from trajdata.augmentation import NoiseHistories

from modules.model_registrar import ModelRegistrar
from models.trajectory_predictor import TrajectoryPredictor

In [15]:
with open('../config/pedestrians.json', 'r', encoding="utf-8") as config_json:
    hyperparams = json.load(config_json)
hyperparams["device"] = "cpu"
hyperparams["edge_encoding"] = True
hyperparams["contrastive_weight"] = 50.0
hyperparams["history_sec"] = 2.8
hyperparams["prediction_sec"] = 4.8
hyperparams["incl_robot_node"] = False
hyperparams["map_encoding"] = False
hyperparams["preprocess_workers"] = 16
hyperparams["trajdata_cache_dir"] = "../data/pedestrian_datasets/.unified_data_cache"

hyperparams["log_p_yt_xz_max"] = 6
hyperparams["single_mode_multi_sample"] = False
hyperparams["single_mode_multi_sample_num"] = 50

# hyperparams

In [16]:
desired_data=[
    "eupeds_eth-train",
]
data_dirs = {
    "eupeds_eth": "~/Projects/trajectron/datasets/eth_ucy_peds",
}

attention_radius = defaultdict(
    lambda: 20.0
)  # Default range is 20m unless otherwise specified.
attention_radius[(AgentType.PEDESTRIAN, AgentType.PEDESTRIAN)] = 5.0

input_noise = 0.0
augmentations = list()
if input_noise > 0.0:
    augmentations.append(NoiseHistories(stddev=input_noise))

In [17]:
batch_size = 4

dataset = UnifiedDataset(
    desired_data=desired_data,
    history_sec=(0.1, hyperparams["history_sec"]),
    future_sec=(0.1, hyperparams["prediction_sec"]),
    agent_interaction_distances=attention_radius,
    incl_robot_future=hyperparams["incl_robot_node"],
    incl_raster_map=hyperparams["map_encoding"],
    only_predict=[AgentType.PEDESTRIAN],
    no_types=[AgentType.UNKNOWN],
    augmentations=augmentations if len(augmentations) > 0 else None,
    num_workers=hyperparams["preprocess_workers"],
    cache_location=hyperparams["trajdata_cache_dir"],
    data_dirs=data_dirs,
    verbose=True,
)

dataloader = data.DataLoader(
    dataset,
    collate_fn=dataset.get_collate_fn(pad_format="right"),
    pin_memory=False if hyperparams["device"] == "cpu" else True,
    batch_size=batch_size,
    shuffle=True,
    num_workers=hyperparams["preprocess_workers"],
    sampler=None,
)

batch: AgentBatch = next(iter(dataloader))

Loading data for matched scene tags: ['zurich-eupeds_eth-train', 'eupeds_eth-train-cyprus']


Calculating Agent Data (Serially): 100%|██████████| 1/1 [00:00<00:00, 6345.39it/s]

1 scenes in the scene index.



Structuring Agent Data Index: 100%|██████████| 1/1 [00:00<00:00, 886.18it/s]


In [18]:
# print(f"Agent Future Length: {batch.agent_fut_len}")
# print(f"Neigbors Future Length: {batch.neigh_fut_len}")

In [19]:
model_dir = "./"
model_registrar = ModelRegistrar(model_dir, hyperparams["device"])

In [20]:
trajectory_predictor = TrajectoryPredictor(
    model_registrar, hyperparams, log_writer=None, device=hyperparams["device"]
)

In [21]:
trajectory_predictor.set_environment()

In [22]:
trajectory_predictor.set_all_annealing_params()

In [23]:
model_registrar.print_model_names()

odict_keys(['PEDESTRIAN/node_history_encoder', 'PEDESTRIAN/node_future_encoder', 'PEDESTRIAN/edge_influence_encoder', 'UNKNOWN->PEDESTRIAN/edge_encoder', 'VEHICLE->PEDESTRIAN/edge_encoder', 'PEDESTRIAN->PEDESTRIAN/edge_encoder', 'BICYCLE->PEDESTRIAN/edge_encoder', 'MOTORCYCLE->PEDESTRIAN/edge_encoder', 'PEDESTRIAN/p_z_x', 'PEDESTRIAN/q_z_xy', 'PEDESTRIAN/decoder/PreGRU', 'PEDESTRIAN/decoder/GRU', 'PEDESTRIAN/decoder/GMM', 'snce'])


In [24]:
lr_scheduler = None
step_scheduler = None
# TODO: define a different set of optimization parameter if updating pre-trined model
optimizer = optim.Adam(
    [
        {
            "params": model_registrar.get_all_but_name_match(
                "map_encoder"
            ).parameters()
        },
        {
            "params": model_registrar.get_name_match("map_encoder").parameters(),
            "lr": hyperparams["map_enc_learning_rate"],
        },
    ],
    lr=hyperparams["learning_rate"],
)
# Set Learning Rate
if hyperparams["learning_rate_style"] == "const":
    lr_scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=1.0)
elif hyperparams["learning_rate_style"] == "exp":
    lr_scheduler = optim.lr_scheduler.ExponentialLR(
        optimizer, gamma=hyperparams["learning_decay_rate"]
    )

if hyperparams["lr_step"] != 0:
    step_scheduler = optim.lr_scheduler.StepLR(
        optimizer, step_size=hyperparams["lr_step"], gamma=0.1
    )

In [25]:
trajectory_predictor.curr_iter = 0
trajectory_predictor.step_all_annealers()

optimizer.zero_grad(set_to_none=True)

train_loss, loss_task, loss_nce = trajectory_predictor(batch)
print(
    f"Total Loss: {train_loss.detach().item():.4f}, Task Loss: {loss_task.item():.4f}, SNCE Loss: {loss_nce.item():.4f}"
)
train_loss.backward()

# Clipping gradients.
if hyperparams["grad_clip"] is not None:
    nn.utils.clip_grad_value_(
        model_registrar.parameters(), hyperparams["grad_clip"]
    )

optimizer.step()

# Stepping forward the learning rate scheduler and annealers.
lr_scheduler.step()

Total Loss: 147.8245, Task Loss: 51.4028, SNCE Loss: 1.9284
